In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import json
import os
import glob
import tf2onnx
import onnx

In [24]:
# SỬA DÒNG NÀY THEO MÁY BẠN NHÉ!!!
BASE_DIR = r"D:\DA2\Project2\bidmc-ppg-and-respiration-dataset-1.0.0"
# Nếu để cùng thư mục với notebook thì dùng:
# BASE_DIR = "bidmc-ppg-and-respiration-dataset-1.0.0"

pattern = os.path.join(BASE_DIR, "bidmc*", "*_Numerics.csv")
csv_files = glob.glob(pattern)

if not csv_files:
    raise FileNotFoundError(f"KHÔNG TÌM THẤY DATASET!\nKiểm tra lại đường dẫn:\n{BASE_DIR}")

print(f"Đã tìm thấy {len(csv_files)} file CSV")
print("Đang đọc 10 file đầu để train nhanh...")

Đã tìm thấy 53 file CSV
Đang đọc 10 file đầu để train nhanh...


In [25]:
dfs = []
for i, file in enumerate(csv_files[:10]):
    print(f"   [{i+1}/10] Đang đọc: {os.path.basename(file)}")
    df = pd.read_csv(file)
    df.columns = df.columns.str.strip()
    if 'HR' in df.columns and 'SpO2' in df.columns:
        clean = df[['HR', 'SpO2']].dropna()
        if len(clean) > 100:
            dfs.append(clean)

data = pd.concat(dfs, ignore_index=True)
print(f"\nHOÀN TẤT! Tổng: {len(data):,} bản ghi hợp lệ")
data.head()

   [1/10] Đang đọc: bidmc_01_Numerics.csv
   [2/10] Đang đọc: bidmc_02_Numerics.csv
   [3/10] Đang đọc: bidmc_03_Numerics.csv
   [4/10] Đang đọc: bidmc_04_Numerics.csv
   [5/10] Đang đọc: bidmc_05_Numerics.csv
   [6/10] Đang đọc: bidmc_06_Numerics.csv
   [7/10] Đang đọc: bidmc_07_Numerics.csv
   [8/10] Đang đọc: bidmc_08_Numerics.csv
   [9/10] Đang đọc: bidmc_09_Numerics.csv
   [10/10] Đang đọc: bidmc_10_Numerics.csv

HOÀN TẤT! Tổng: 4,791 bản ghi hợp lệ


,HR,SpO2
0,94,97.0
1,94,97.0
2,94,97.0
3,92,97.0
4,93,97.0


In [26]:
def classify_health(hr, spo2):
    if 60 <= hr <= 100 and spo2 >= 95: return 'Bình thường'
    if 100 < hr <= 120 and spo2 >= 94: return 'Hoạt động nhẹ'
    if 120 < hr <= 140 and spo2 >= 92: return 'Hoạt động trung bình'
    if hr > 140 and spo2 >= 90: return 'Hoạt động mạnh'
    if hr < 60: return 'Cảnh báo sức khỏe không ổn định'
    if hr > 160 or spo2 < 85: return 'Nhịp tim cao bất thường'
    if 50 <= hr <= 70 and spo2 >= 97: return 'Sức khỏe siêu tốt'
    if spo2 < 80 or hr > 180: return 'Cảnh báo nguy hiểm'
    return 'Không xác định'

print("Đang gắn nhãn...")
data['label'] = data.apply(lambda x: classify_health(x['HR'], x['SpO2']), axis=1)
data = data[data['label'] != 'Không xác định']

labels = ['Bình thường', 'Hoạt động nhẹ', 'Hoạt động trung bình', 'Hoạt động mạnh',
          'Cảnh báo sức khỏe không ổn định', 'Nhịp tim cao bất thường',
          'Sức khỏe siêu tốt', 'Cảnh báo nguy hiểm']

print("\nPhân bố nhãn:")
print(data['label'].value_counts())

Đang gắn nhãn...

Phân bố nhãn:
label
Bình thường      3339
Hoạt động nhẹ       8
Name: count, dtype: int64


In [27]:
y = data['label'].map({l: i for i, l in enumerate(labels)}).values
X = data[['HR', 'SpO2']].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}")
print("Dữ liệu đã sẵn sàng!")

Train: 2,677 | Test: 670
Dữ liệu đã sẵn sàng!


In [28]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(len(labels), activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Bắt đầu train (40 epochs)...")
history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=128,
    validation_data=(X_test, y_test),
    verbose=1
)

Bắt đầu train (40 epochs)...
Epoch 1/40
21/21 [==============================] - 1s 15ms/step - loss: 1.9161 - accuracy: 0.3381 - val_loss: 1.6252 - val_accuracy: 0.9836
Epoch 2/40
21/21 [==============================] - 0s 5ms/step - loss: 1.3070 - accuracy: 0.9458 - val_loss: 0.8713 - val_accuracy: 0.9970
Epoch 3/40
21/21 [==============================] - 0s 5ms/step - loss: 0.5495 - accuracy: 0.9978 - val_loss: 0.2187 - val_accuracy: 0.9970
Epoch 4/40
21/21 [==============================] - 0s 7ms/step - loss: 0.1416 - accuracy: 0.9978 - val_loss: 0.0617 - val_accuracy: 0.9970
Epoch 5/40
21/21 [==============================] - 0s 5ms/step - loss: 0.0539 - accuracy: 0.9978 - val_loss: 0.0405 - val_accuracy: 0.9970
Epoch 6/40
21/21 [==============================] - 0s 5ms/step - loss: 0.0375 - accuracy: 0.9978 - val_loss: 0.0353 - val_accuracy: 0.9970
Epoch 7/40
21/21 [==============================] - 0s 7ms/step - loss: 0.0307 - accuracy: 0.9978 - val_loss: 0.0329 - val_accurac

In [29]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nĐỘ CHÍNH XÁC TRÊN TẬP TEST: {test_acc*100:.2f}%")

# Lưu scaler
with open('scaler.json', 'w', encoding='utf-8') as f:
    json.dump({
        'mean': scaler.mean_.tolist(),
        'scale': scaler.scale_.tolist()
    }, f)
print("Đã lưu scaler.json")


ĐỘ CHÍNH XÁC TRÊN TẬP TEST: 99.70%
Đã lưu scaler.json


In [30]:
ONNX_PATH = "hr_spo2_model.onnx"
print(f"Đang convert model sang ONNX → {ONNX_PATH}...")

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=[tf.TensorSpec([None, 2], tf.float32, name="input")],
    opset=13
)

onnx.save(onnx_model, ONNX_PATH)
print("CONVERT THÀNH CÔNG 100%!")
print(f"File đã tạo: {os.path.abspath(ONNX_PATH)}")

Đang convert model sang ONNX → hr_spo2_model.onnx...
CONVERT THÀNH CÔNG 100%!
File đã tạo: d:\DA2\Project2\hr_spo2_model.onnx
